In [109]:
from pyspark.context import SparkContext
'''Random Forest'''
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.linalg import Vectors
from pyspark.sql import DataFrame

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils

In [110]:
sc = SparkContext.getOrCreate()
data_path = "./properties_coor.csv"
data_set = spark.read.options(header="true", parserLib="univocity", inferSchema="true").csv(data_path)
print data_set.schema
headers = data_set.columns
# print(headers)
print 'headers names and types: \n'
print data_set.dtypes
print data_set.count()

StructType(List(StructField(_c0,IntegerType,true),StructField(Parcel ID,StringType,true),StructField(Lat,DoubleType,true),StructField(Lng,DoubleType,true),StructField(Property address,StringType,true),StructField(Tax rate area,IntegerType,true),StructField(Neighborhood,IntegerType,true),StructField(Lot sq. ft.,IntegerType,true),StructField(Property class,IntegerType,true),StructField(Year built,IntegerType,true),StructField(Square feet,IntegerType,true),StructField(Rooms,DoubleType,true),StructField(Bedrooms,DoubleType,true),StructField(Bathrooms,DoubleType,true),StructField(ZIP code,IntegerType,true),StructField(Median Price(2017),IntegerType,true),StructField(Median $ per Sq. Ft.(2017),IntegerType,true),StructField(Value at Median $ per Sq. Ft.(2017),DoubleType,true)))
headers names and types: 

[('_c0', 'int'), ('Parcel ID', 'string'), ('Lat', 'double'), ('Lng', 'double'), ('Property address', 'string'), ('Tax rate area', 'int'), ('Neighborhood', 'int'), ('Lot sq. ft.', 'int'), ('Pr

In [111]:
data_set.take(5)

[Row(_c0=0, Parcel ID=u'429-01-002', Lat=37.3084232, Lng=-121.8951796, Property address=u'1246 Curtiss Av San Jose, CA 95125', Tax rate area=17108, Neighborhood=0, Lot sq. ft.=13200, Property class=0, Year built=1900, Square feet=1327, Rooms=8.0, Bedrooms=4.0, Bathrooms=2.0, ZIP code=95125, Median Price(2017)=1250000, Median $ per Sq. Ft.(2017)=713, Value at Median $ per Sq. Ft.(2017)=757539.2),
 Row(_c0=1, Parcel ID=u'429-01-003', Lat=37.3085298, Lng=-121.895274, Property address=u'1232 Curtiss Av San Jose, CA 95125', Tax rate area=17108, Neighborhood=0, Lot sq. ft.=12200, Property class=0, Year built=1947, Square feet=1575, Rooms=7.0, Bedrooms=3.0, Bathrooms=2.0, ZIP code=95125, Median Price(2017)=1250000, Median $ per Sq. Ft.(2017)=713, Value at Median $ per Sq. Ft.(2017)=899113.6),
 Row(_c0=2, Parcel ID=u'429-01-005', Lat=37.3088046, Lng=-121.8955309, Property address=u'1220 Curtiss Av San Jose, CA 95125', Tax rate area=17108, Neighborhood=0, Lot sq. ft.=18564, Property class=0, Ye

In [112]:
new_data = data_set.groupBy(headers[4]).count().collect()

In [113]:
len(new_data)

19793

In [114]:
type(data_set)
data_set = reduce(lambda x,y: x.drop(y),[headers[1], headers[4]],data_set)
# data_set = data_set[0] + data_set[2:]
print data_set.dtypes
# data_set = data_set.drop(headers[4]).collect()
# print data_set.dtype()

[('_c0', 'int'), ('Lat', 'double'), ('Lng', 'double'), ('Tax rate area', 'int'), ('Neighborhood', 'int'), ('Lot sq. ft.', 'int'), ('Property class', 'int'), ('Year built', 'int'), ('Square feet', 'int'), ('Rooms', 'double'), ('Bedrooms', 'double'), ('Bathrooms', 'double'), ('ZIP code', 'int'), ('Median Price(2017)', 'int'), ('Median $ per Sq. Ft.(2017)', 'int'), ('Value at Median $ per Sq. Ft.(2017)', 'double')]


In [115]:
transformed_df = data_set.rdd.map(lambda row: LabeledPoint(row[-3], Vectors.dense(row[:-3]+row[-2:])))
trained_data, tested_data = transformed_df.randomSplit([0.7, 0.3], seed = 100)

In [116]:
trained_data.take(50)
# print tested_data.count()

[LabeledPoint(1250000.0, [0.0,37.3084232,-121.8951796,17108.0,0.0,13200.0,0.0,1900.0,1327.0,8.0,4.0,2.0,95125.0,713.0,757539.2]),
 LabeledPoint(1250000.0, [3.0,37.3089809,-121.8956939,17108.0,0.0,5040.0,0.0,1905.0,1044.0,5.0,3.0,1.0,95125.0,713.0,595984.0]),
 LabeledPoint(1250000.0, [4.0,37.3095539,-121.896306,17108.0,0.0,3300.0,0.0,1926.0,960.0,5.0,2.0,1.0,95125.0,713.0,548031.2]),
 LabeledPoint(1250000.0, [6.0,37.3081991,-121.895019,17108.0,0.0,8500.0,0.0,1989.0,2581.0,10.0,4.0,2.5,95125.0,713.0,1473404.8]),
 LabeledPoint(1250000.0, [7.0,37.3083563,-121.8948274,17108.0,0.0,8500.0,0.0,1988.0,2371.0,7.0,3.0,2.5,95125.0,713.0,1353523.2]),
 LabeledPoint(1250000.0, [8.0,37.30863,-121.8936671,17108.0,0.0,6300.0,0.0,2006.0,2122.0,6.0,3.0,3.0,95125.0,713.0,1514222.0]),
 LabeledPoint(1250000.0, [9.0,37.308529,-121.89352,17108.0,0.0,5000.0,0.0,1922.0,848.0,6.0,2.0,1.0,95125.0,713.0,484094.4]),
 LabeledPoint(1250000.0, [10.0,37.3084411,-121.8934057,17108.0,0.0,5000.0,0.0,1939.0,1093.0,5.0,2.0,1

In [117]:
# assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericColumns
# assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# stages += [assembler]

In [118]:
labeled_price = headers[15]
rf = RandomForest.trainRegressor(trained_data, categoricalFeaturesInfo={},
                                    numTrees=10, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=4, maxBins=32)
predictions = rf.predict(tested_data.map(lambda x: x.features))
labelsAndPredictions = tested_data.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(tested_data.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression forest model:')
print(rf.toDebugString())

Test Mean Squared Error = 16570322186.7
Learned regression forest model:
TreeEnsembleModel regressor with 10 trees

  Tree 0:
    If (feature 0 <= 9802.0)
     If (feature 5 <= 3060.0)
      If (feature 12 <= 95118.0)
       If (feature 8 <= 1404.0)
        Predict: 581985.2941176471
       Else (feature 8 > 1404.0)
        Predict: 460000.0
      Else (feature 12 > 95118.0)
       If (feature 7 <= 1954.0)
        Predict: 1210000.0
       Else (feature 7 > 1954.0)
        Predict: 761344.0233236152
     Else (feature 5 > 3060.0)
      If (feature 1 <= 37.3121024)
       If (feature 4 <= 2.0)
        Predict: 1224530.0343249429
       Else (feature 4 > 2.0)
        Predict: 1097140.7692307692
      Else (feature 1 > 37.3121024)
       If (feature 12 <= 95112.0)
        Predict: 670000.0
       Else (feature 12 > 95112.0)
        Predict: 1250000.0
    Else (feature 0 > 9802.0)
     If (feature 12 <= 95122.0)
      If (feature 6 <= 0.0)
       If (feature 3 <= 17108.0)
        Predict: 

In [120]:
# Save and load model
# rf.save(sc, "myrfModel")
sameModel = RandomForestModel.load(sc, "myrfModel")

In [121]:
prediction = rf.predict([60.0,37.307741,-121.89593,17108.0,0.0,8400.0,0.0,1938.0,1320.0,6.0,2.0,1.0,95125.0,713.0,753543.2])
print prediction

1239115.60373
